# Hull-White Model To ATM Caplet Market Implied Volatilities

# Requirements



Your objective will be to calibrate a Hull-White model to ATM caplet market implied volatilities. You will be expected to go through the following steps: 
1. download ATM cap data, 
2. program in Matlab or Python a theoretical closed-form pricing function for the Hull-White model, 
3. program a Monte-Carlo simulation-based function and 
4. verify that it produces the same price of all caps as the closed-form solution, and 
5. calibrate the Hull-White model to the data by minimizing the sum of squared pricing errors over model parameters. 

Your final project deliverables should include clean, bug-free, and well-commented Matlab source code accompanied by “Executive Summary” of results with tables, graphs comparing model vs. market implied vols, as well as intelligent comments on model limitations.

Performing well on the group project requires you to write an “Executive Summary” that discusses your modeling techniques and results. Ideally, you should be turning in a professional, well written report that details the motivation for the project, how you implemented the project, and how your results match up to the observed market quantities. While there are no strict requirements for what your executive report should contain, it is suggested that at a minimum the report should contain a well-thought discussion of how you implemented the project, the model limitations, and how well your model fits the data. The remaining choice of content is completely up to you and your group to decide. Please note that this open structure of what you submit for the project enables you to exercise a high degree of flexibility in implementing your results. Thus, creativity is an integral part of the group. Making modeling assumptions is completely valid, and all you need to do is clearly state any assumptions you make and why they are justifiable.

Please keep the executive report to 15 pages or less and only include discussions and graphs. Points will be deducted for including code in the executive report.


In [2]:
from scipy.stats import norm
from math import exp, sqrt, log
import scipy.io
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [3]:
data_path = '../data/'
cap = pd.read_csv(data_path + 'cap.csv')

In [4]:
cap

,CapStrike,CapVols,Days,Discount,Expiry,HedgeRatio,IntrinsicPV,Notional,PV,PayDate,Payment,ResetRate,T_i,T_iM1,TimePV,tau_i
0,2.71212,9.82,92,0.987013,07/16/2019,0.17,0.00,10000000,223.73,10/18/2019,226.67,2.57190,0.513889,0.252778,223.73,0.261111
1,2.71212,9.52,95,0.980422,10/16/2019,0.20,0.00,10000000,431.15,01/21/2020,439.76,2.54782,0.777778,0.508333,431.15,0.269444
2,2.71212,9.22,90,0.974283,01/17/2020,0.20,0.00,10000000,491.17,04/20/2020,504.14,2.51966,1.027778,0.766667,491.17,0.261111
3,2.71212,16.23,91,0.968321,04/16/2020,0.28,0.00,10000000,1547.25,07/20/2020,1597.87,2.43567,1.280556,1.016667,1547.25,0.263889
4,2.71212,16.72,91,0.962563,07/16/2020,0.26,0.00,10000000,1557.72,10/19/2020,1618.30,2.36663,1.533333,1.269444,1557.72,0.263889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2.71212,24.74,90,0.448040,01/17/2048,0.34,241.19,10000000,15154.69,04/20/2048,33824.44,2.73365,29.436111,29.175000,14913.50,0.261111
115,2.71212,24.78,91,0.444970,04/16/2048,0.33,195.30,10000000,15259.08,07/20/2048,34292.41,2.72948,29.688889,29.425000,15063.78,0.263889
116,2.71212,24.82,91,0.441925,07/16/2048,0.33,147.29,10000000,15195.79,10/19/2048,34385.43,2.72530,29.941667,29.677778,15048.50,0.263889
117,2.71212,24.86,92,0.438873,10/15/2048,0.32,99.67,10000000,15297.01,01/19/2049,34855.17,2.72100,30.197222,29.930556,15197.34,0.266667


Price Data
- `CapStrike`: Caplet Strike Price $K$
- `CapVols`(%): Caplet Volatility $\sigma$
- `IntrinsicPV`: $K - K_{ATM}$
- `Discount`: Zero-coupon Bond $P(t, S)$
- `Notional`: Notional Price $N$

Time Data
- `Expiry`: Date of Expiry $T$
  - `T_iM1`: Conversion of the Amount of Time between Expiry Date($T_i$) and Today($t=0$) (according to the Time Convention) $T_i - t$
  - actually it's last term's PayDate, so $S_{i-1} = T_i$
- `PayDate`: Date of Payoff $S$
  - `T_i`: Conversion of the Amount of Time between PayDate($S_i$) and Today($t=0$) (according to the Time Convention)
- `Days`: Number of Days between $T$ and $S$
  - `tau_i`: $\tau_i = S_i - T_i$
- `T_i` - `T_iM1` = `tau_i`

Today($t=0$): 20190416

# Functions

## Function 1.X

**Definition 1.2.1. Zero-coupon bond**. 

A T-maturity zero-coupon bond (pure discount bond) is a contract that guarantees its holder the payment of one unit of currency at time $T$, with no intermediate payments. The contract value at time $t<T$ is denoted by $P(t, T)$. Clearly, $P(T, T)=1$ for all $T$

**Definition 1.4.1. Simply-compounded forward interest rate.**

The simply-compounded forward interest rate prevailing at time $t$ for the expiry $T>t$ and maturity $S>T$ is denoted by $F(t ; T, S)$ and is defined by
$$
F(t ; T, S):=\frac{1}{\tau(T, S)}\left(\frac{P(t, T)}{P(t, S)}-1\right)
$$

$$
\mathbf{C a p}^{\text {Black }}\left(0, \mathcal{T}, \tau, N, K, \sigma_{\alpha, \beta}\right)=N \sum_{i=\alpha+1}^\beta P\left(0, T_i\right) \tau_i \operatorname{Bl}\left(K, F\left(0, T_{i-1}, T_i\right), v_i, 1\right)
$$

where, denoting by $\Phi$ the standard Gaussian cumulative distribution function,
$$
\operatorname{Bl}(K, F, v, \omega), 
=F \omega \Phi\left(\omega d_1(K, F, v)\right)-K \omega \Phi\left(\omega d_2(K, F, v)\right)
$$

$$
\begin{aligned}
d_1(K, F, v) & =\frac{\ln (F / K)+v^2 / 2}{v} \\
d_2(K, F, v) & =\frac{\ln (F / K)-v^2 / 2}{v} \\
v_i & =\sigma_{\alpha, \beta} \sqrt{T_{i-1}}
\end{aligned}
$$

In [5]:
def black_cap_pricing(sigma_input=None, cap=cap):
    """Convert Bloomberg Market Implied Vol into Dollar Price - Formula 1.26

    Args:
        cap (_type_, optional): _description_. Defaults to cap.

    Returns:
        _type_: _description_
    """
    result_cap = 0
    N = cap['Notional'][0]
    for i in range(1, len(cap)): 
        # zero-coupon bond price at time t
        P_t_T = cap['Discount'][i - 1]  # T is the expiry time, T_i = S_i-1
        P_t_S = cap['Discount'][i]  # S is the maturity time, S_i
        
        tau_i = cap['tau_i'][i]
        # simply-compounded forward rate
        F_t_T_S = (P_t_T / P_t_S - 1) / tau_i
        # strike price
        K = cap['CapStrike'][i]
        # volatility
        if sigma_input is None:
            sigma = cap['CapVols'][i] / 100
        else:
            sigma = sigma_input
        omega = 1
        v = sigma * sqrt(cap['T_iM1'][i])
        d1 = (log(F_t_T_S / K) + 0.5 * v**2) / v
        d2 = d1 - v
        Bl = omega * (F_t_T_S * norm.cdf(omega * d1) - K * norm.cdf(omega * d2))
        result_cap += Bl * tau_i * P_t_S
        
    return result_cap * N


black_cap_pricing(cap=cap)

391.5745340067972

Caplet

$$
\mathbf{C p l}\left(t, t_{i-1}, t_i, \tau_i, N, X\right)=N_i^{\prime} \mathbf{Z B P}\left(t, t_{i-1}, t_i, X_i^{\prime}\right)
$$
where
$$
\begin{aligned}
X_i^{\prime}, =\frac{1}{1+X \tau_i} \\
N_i^{\prime}, =N\left(1+X \tau_i\right)
\end{aligned}
$$

In [10]:
def ZBP(i, X, cap=cap):
    # zero-coupon bond price at time t
    # P_t_T = cap['Discount'][i - 1]  # T is the expiry time, T_{i-1}
    # P_t_S = cap['Discount'][i]  # S is the maturity time, T_i
    # P_T_S = P_t_S / P_t_T
    # return P_t_T * max(X - P_T_S, 0)
    pass

def HW_caplets(i, N, X, cap=cap):
    """HW model, Caplets price, based on ZB put Pricing - Formula 2.27

    Args:
    """
    X_prime = 1 / (1 + X * cap['tau_i'][i])
    N_prime = N * (1 + X * cap['tau_i'][i])
    return N_prime * ZBP(i, cap['T_iM1'][i - 1], cap['T_i'][i], X_prime, cap=cap)

Converted the optimized HW Cap prices into Black Implied Vols

In [11]:
def Price_to_Vol(price, cap=cap, tol=0.0001):
    """
    Converted the optimized HW Cap prices into Black Implied Vols

    Args:
        price (float): Option Price

    Returns:
        float: Implied Volatility
    """
    # initial guess
    sigma = 0.01
    sigma = minimize(fun=black_cap_pricing - price,
                     x0=sigma, args=(sigma, cap), tol=tol).x
    # while True:
    #     price_diff = black_cap_pricing(cap=cap) - price
    #     if abs(price_diff) < tol:
    #         break
    #     vol -= price_diff / (S * norm.pdf((log(S/K) + (r + 0.5 * vol**2) * T) / (vol * sqrt(T))))
    return sigma

## Function 2.X

$$
\mathbf{Z B P}(t, T, S, X)=P(t, T) E^T\left((X - P(T, S))^{+} \mid \mathcal{F}_t\right)
$$

# Executive Summary